In [1]:
import sys

sys.path.append('../GraphStructureLearning')

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
import pickle

from torch_geometric.data import Data as gData
from torch_geometric.utils import to_networkx, to_undirected
from torch_geometric.nn import MessagePassing

import networkx as nx
import matplotlib.pyplot as plt

from torch_geometric_temporal.nn.recurrent import DCRNN

# GTS

In [3]:
from models.GTS.gts_graph_learning import GTS_Graph_Learning
from models.GTS.gts_forecasting_module import GTS_Forecasting_Module
from models.GTS.DCRNN import DCRNN
from utils.utils import build_fully_connected_edge_idx

from glob import glob
import yaml
from easydict import EasyDict as edict

In [4]:
config_file = glob('./config/GTS/*.yaml')[0]
config = edict(yaml.load(open(config_file, 'r'), Loader=yaml.FullLoader))

In [5]:
# node_feas = torch.rand(config.nodes_num, config.node_features, 1000)
# edge_index = build_fully_connected_edge_idx(num_nodes=config.nodes_num)

In [6]:
spike = pickle.load(open('./data/LNP_spk_all.pickle', 'rb'))

spike = torch.FloatTensor(spike[:,:1000])

In [7]:
config.nodes_num = 100

In [14]:
edge_index = build_fully_connected_edge_idx(num_nodes=config.nodes_num)

In [21]:
edge_index.shape

torch.Size([2, 9900])

In [22]:
spike.shape

torch.Size([100, 1000])

In [12]:
gl = GTS_Graph_Learning(config)

In [15]:
adj = gl(spike, edge_index)

In [18]:
z_1 = F.gumbel_softmax(adj, tau=0.3, hard=True)
z_1 = torch.transpose(z_1, 0, 1)

In [19]:
edge_ = []

for ii, rel in enumerate(z_1[0]):
    if bool(rel):
        edge_.append(edge_index[:, ii])

adj_matrix = torch.stack(edge_, dim=-1)

In [20]:
adj_matrix.shape

torch.Size([2, 3867])

In [23]:
config.num_layer=2

In [24]:
config.embedding_dim = 13
config.in_channels = 1

In [25]:
encoder_dcrnn = DCRNN(config)
decoder_dcrnn = DCRNN(config)

In [27]:
inputs = spike.reshape(-1,100,1,100)
inputs.shape

torch.Size([10, 100, 1, 100])

In [28]:
encoder_hidden_state = encoder_dcrnn(inputs[0,:,:,0], adj_matrix)

In [29]:
encoder_hidden_state_2 = encoder_dcrnn(inputs[0,:,:,1], adj_matrix, hidden_state=encoder_hidden_state)

In [30]:
decoder_hidden_state = decoder_dcrnn(inputs[0,:,:,3], adj_matrix, hidden_state=encoder_hidden_state_2)

In [31]:
decoder_hidden_state[-1].shape

torch.Size([100, 64])

In [32]:
forecasting = GTS_Forecasting_Module(config)

In [33]:
encoder_inputs = torch.rand(config.nodes_num, config.node_features, 70)
decoder_inputs = torch.rand(config.nodes_num, config.node_features, 30)